In [8]:
import  numpy as np
import  matplotlib.pyplot as plt
import cv2
import pandas as pd
import requests
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import normalize
import pickle

In [2]:
# Define the pre-trained ResNet model
resnet = models.resnet50(pretrained=True)

# Remove the last layer
feature_extractor = torch.nn.Sequential(*(list(resnet.children())[:-1]))


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:


# Load the dataset back
with open('data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

    

data=loaded_data
tfidf = pd.read_csv('tfidf.csv')
tfidf_df = pd.DataFrame(tfidf)
# normalise tfidf scores
normalized_tfidf = normalize(tfidf, axis=1, norm='l2')
# convert normalized tfidf scores to a dataframe
normalized_tfidf_df = pd.DataFrame(normalized_tfidf)



imgSimilarity=pd.read_csv('image_similarity.csv')
image_similarity_M = pd.DataFrame(imgSimilarity)



In [19]:


def adjust_brightness_and_contrast(image, brightness=0, contrast=0):
        # Alpha controls contrast; Beta controls brightness.
        alpha = 1 + contrast / 127
        beta = brightness
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return adjusted_image

def preprocess(image_url):
    response = requests.get(image_url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)
    if image is None:
        return None
    image_height, image_width = image.shape[:2]
    #border removal
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    image = image[y:y+h,x:x+w]
    image = cv2.resize(image, (80, 80))


    # Random flip
    flip_direction = np.random.choice(["horizontal", "vertical"])
    if flip_direction == "horizontal":
        image = cv2.flip(image, 1)  # 1: Flip horizontally
    elif flip_direction == "vertical":
        image = cv2.flip(image, 0)  # 0: Flip vertically

    

    #pixel normalisation
    def normalize_image(image):
        normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        return normalized_image

    image = normalize_image(image)
    

    #brightness and contrast adjustment
    def calculate_brightness_and_contrast(image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean, std_dev = cv2.meanStdDev(gray_image)
        return mean[0][0], std_dev[0][0]

    

    mean_brightness, std_dev_contrast = calculate_brightness_and_contrast(image)

    # Set a threshold for deciding whether to adjust brightness and contrast
    brightness_threshold = 20  
    contrast_threshold = 10 

    # Adjust brightness and contrast only if needed
    if mean_brightness < brightness_threshold or std_dev_contrast < contrast_threshold:
        brightness = 20  
        contrast = 20  
        adjusted_image = adjust_brightness_and_contrast(image, brightness=brightness, contrast=contrast)
    else:
        adjusted_image = image

    return adjusted_image





def extract_features(image):
    img = cv2.resize(image, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = transforms.ToTensor()(img)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img_normalized = normalize(img_tensor)
    img_normalized = img_normalized.unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(img_normalized)
    features = features.view(features.size(0), -1)
    features = features.numpy()
    return features




def ImagecosineSim(image_id1, image_id2):
    features1 = imageTOvec(image_id1)
    features2 = imageTOvec(image_id2)
    similarity = np.dot(features1, features2.T)
    return similarity



def imageTOvec(image_id):
    image_url = image_id
    # print(image_url)
    image = preprocess(image_url)
    if image is None:
        return None
    features = extract_features(image)
    return features

In [20]:
#  preprorcess revierw texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer as ps

# Download the stopwords corpus if you haven't already
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def preprocess_review_text(review_text):
    # print(review_text)
    # print index of the reivew text
    review_text = str(review_text)
    review_text = review_text.lower()
    review_text = re.sub(r'[^\w\s]', '', review_text)
    # remove stop words
    review_text = ' '.join([word for word in review_text.split() if word not in stop_words])
    # do stemming
    review_text = ' '.join([ps().stem(word) for word in review_text.split()])
    return review_text



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
import numpy as np
from collections import Counter
import math

def tokenize(text):
    # Split text into tokens (words)
    return text.split()

def calculate_tf(text):
    # Calculate term frequency (TF) for each word in the text
    tokens = tokenize(text)
    word_count = Counter(tokens)
    total_words = len(tokens)
    tf = {word: count / total_words for word, count in word_count.items()}
    return tf

def calculate_idf(documents):
    # Calculate inverse document frequency (IDF) for each word
    total_documents = len(documents)
    all_words = set([word for document in documents for word in tokenize(document)])
    idf = {}
    for word in all_words:
        doc_count = sum([1 for document in documents if word in tokenize(document)])
        idf[word] = math.log10(total_documents / doc_count)
    return idf

def calculate_tfidf(text, idf):
    # Calculate TF-IDF for each word in the text using precomputed IDF values
    tf = calculate_tf(text)
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf




In [32]:


def cosineSimText(text1, text2):

    idf = calculate_idf([text1, text2])
    tfidf1 = calculate_tfidf(text1, idf)
    tfidf2 = calculate_tfidf(text2, idf)

    # Calculate the dot product of tfidf1 and tfidf2
    dot_product = sum([tfidf1[word] * tfidf2[word] for word in tfidf1 if word in tfidf2])

    # Calculate the magnitude of tfidf1 and tfidf2
    magnitude1 = math.sqrt(sum([tfidf1[word]**2 for word in tfidf1]))
    magnitude2 = math.sqrt(sum([tfidf2[word]**2 for word in tfidf2]))

    # Calculate the cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity


In [36]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.35204768, 0.53676, 0.49917382, 0.42068002..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.35107985, 0.5320194, 0.4778194, 0.4419130..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38004225, 0.5307589, 0.4896015, 0.4049801..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.34581208, 0.55054736, 0.5456345, 0.385915..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.3345537, 0.50541735, 0.48309487, 0.465147..."


In [33]:
tfidf_df.head()

,love,vintag,spring,strat,good,tension,great,stabil,float,bridg,...,mayer,importantli,toneprint,stringthru,stopflair,biggi,accord,screenshot,amazoncom,piti
0,0.060206,0.23263,0.28732,0.091102,0.045883,0.139794,0.032990,0.139794,0.168192,0.097901,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.00000,0.00000,0.000000,0.029924,0.000000,0.021515,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.00000,0.00000,0.000000,0.057354,0.000000,0.041238,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.010758,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
def cosine_similarityTF(dic1, dict2):
    # Calculate the dot product
    dot_product = sum([dic1[word] * dict2[word] for word in dic1 if word in dict2])

    # Calculate the magnitude of the vectors
    magnitude1 = math.sqrt(sum([dic1[word]**2 for word in dic1]))
    magnitude2 = math.sqrt(sum([dict2[word]**2 for word in dict2]))

    # Calculate the cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity

In [60]:
def most_similar_reviews(query_text):
    # Preprocess the query text
    preprocessed_query = preprocess_review_text(query_text)

    #  drop all the words from preprocessed_query that are not in the tfidf_df
    preprocessed_query = ' '.join([word for word in preprocessed_query.split() if word in tfidf_df.columns])

    # convert the query text to a tfidf vector
    query_tfidf = calculate_tfidf(preprocessed_query, calculate_idf(data['Review Text']))

    print(preprocessed_query)
    # find and print cosine similarity between the query text and 1st review
    #  retrieve tfidf_df.iloc[0] adn change it into  a dictionary
    tfidf_dict = tfidf_df.iloc[0].to_dict()

    similarity = cosine_similarityTF(tfidf_dict, query_tfidf)
    print()

    # find and print cosine similarity between the query text and all reviews and rank them
    similarities = tfidf_df.apply(lambda row: cosine_similarityTF(row.to_dict(), query_tfidf), axis=1)
    print("similarities::::")
    print(similarities)
    #  use similarity to rank reviews in descending order
    similars=[]
    for i in range(len(similarities)):
        similars.append((i, similarities[i]))
    similars.sort(key=lambda x: x[1], reverse=True)
    print(similars)
    #  return the most similar reviews
    most_similar = data.iloc[[x[0] for x in similars]]


    return most_similar


#  find the most similar reviews to the query text
query_text ="""
use everyth acoust bass ukulel rando wantd
"""
most_similar = most_similar_reviews(query_text)



use everyth acoust bass ukulel

similarities::::
0      0.000000
1      0.000000
2      0.222974
3      0.000000
4      0.197021
         ...   
995    0.000000
996    0.000000
997    0.005887
998    0.007303
999    0.016769
Length: 1000, dtype: float64
[(521, 0.3185190433750043), (526, 0.2993913054993825), (137, 0.2543392547953803), (786, 0.24498054996443322), (202, 0.23351073775895737), (692, 0.23156980046764178), (432, 0.2272936154387274), (402, 0.22441371076023228), (2, 0.22297415582961394), (278, 0.21128074922138768), (888, 0.20042539343924787), (4, 0.19702084172731324), (427, 0.1968077567307111), (895, 0.18913971903569707), (564, 0.18145407707781214), (119, 0.16855693162476237), (648, 0.1665986027671351), (926, 0.16588934817498058), (793, 0.16301166734067887), (769, 0.15973828593872755), (59, 0.1557478087432324), (149, 0.15317247429228528), (660, 0.15129771567363617), (288, 0.14855286735679182), (147, 0.14786654728578533), (457, 0.1461560544498011), (464, 0.1461302729741693), (70

IndexError: positional indexers are out-of-bounds

In [45]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.35204768, 0.53676, 0.49917382, 0.42068002..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.35107985, 0.5320194, 0.4778194, 0.4419130..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38004225, 0.5307589, 0.4896015, 0.4049801..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.34581208, 0.55054736, 0.5456345, 0.385915..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.3345537, 0.50541735, 0.48309487, 0.465147..."
